In [6]:
import re


def transform_to_fasta(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            # Split the line into components
            parts = line.strip().split(' : ')
            if len(parts) != 4:
                continue  # Skip invalid lines

            # Extract relevant information
            pdb_id = parts[0]
            sequence = parts[3]

            # Write in FASTA format
            outfile.write(f'>{pdb_id}\n')

            # Write sequence in lines of 100 characters
            for i in range(0, len(sequence), 100):
                outfile.write(sequence[i:i+100] + '\n')


# Usage
input_file = '../bosutinib/bosutinib_binding.txt'
output_file = '../bosutinib/bosutinib_binding.fasta'
transform_to_fasta(input_file, output_file)

print(f"Transformation complete. FASTA file saved as {output_file}")

Transformation complete. FASTA file saved as ../bosutinib/bosutinib_binding.fasta
